In [0]:
df=spark.read.option("multiline","true").json("/FileStore/tables/carsdata.json")
df.show(truncate=False)

+---+----------------------------------------+----+
|age|cars                                    |name|
+---+----------------------------------------+----+
|30 |[{Ford, null, Mustang}, {null, BMW, X5}]|John|
+---+----------------------------------------+----+



In [0]:
from pyspark.sql.functions import explode
df2=df.select(df.age,explode(df.cars),df.name)
display(df2)

age,col,name
30,"List(Ford, null, Mustang)",John
30,"List(null, BMW, X5)",John


In [0]:
df_flat=df2.select(df2.age,df2.name,df2.col.car1,df2.col.car2,df2.col.model)
display(df_flat)

age,name,col.car1,col.car2,col.model
30,John,Ford,null,Mustang
30,John,null,BMW,X5


In [0]:
df_flat=df2.select(df2.age,df2.name,df2.col.car1.alias('car1'),df2.col.car2.alias('car2'),df2.col.model.alias('model'))
display(df_flat)

age,name,car1,car2,model
30,John,Ford,null,Mustang
30,John,null,BMW,X5


In [0]:
orders_df=spark.read.option("multiline",True).json('/FileStore/tables/orders_data.json')
orders_df.printSchema()

root
 |-- datasets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- customerId: string (nullable = true)
 |    |    |-- orderDate: string (nullable = true)
 |    |    |-- orderDetails: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- productId: string (nullable = true)
 |    |    |    |    |-- quantity: long (nullable = true)
 |    |    |    |    |-- sequence: long (nullable = true)
 |    |    |    |    |-- totalPrice: struct (nullable = true)
 |    |    |    |    |    |-- gross: long (nullable = true)
 |    |    |    |    |    |-- net: long (nullable = true)
 |    |    |    |    |    |-- tax: long (nullable = true)
 |    |    |-- orderId: string (nullable = true)
 |    |    |-- shipmentDetails: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- postalCode: string (nullable = true)
 |    |    

In [0]:
display(orders_df)

datasets,filename
"List(List(cust5001, 2021-12-24 00.00.00.000, List(List(prd9001, 2, 1, List(550, 500, 50)), List(prd9002, 3, 2, List(300, 240, 60))), ord1001, List(Delhi, India, 110040, New Delhi, M.G.Road)), List(cust5002, 2021-12-25 00.00.00.000, List(List(prd9001, 1, 1, List(275, 250, 25)), List(prd9004, 4, 2, List(1000, 900, 100))), ord1002, List(Mumbai, India, 400064, Maharastra, Malad)))",orderDetails


In [0]:
orders_df2=orders_df.select(orders_df.filename,explode(orders_df.datasets).alias('orders'))
display(orders_df2)

filename,orders
orderDetails,"List(cust5001, 2021-12-24 00.00.00.000, List(List(prd9001, 2, 1, List(550, 500, 50)), List(prd9002, 3, 2, List(300, 240, 60))), ord1001, List(Delhi, India, 110040, New Delhi, M.G.Road))"
orderDetails,"List(cust5002, 2021-12-25 00.00.00.000, List(List(prd9001, 1, 1, List(275, 250, 25)), List(prd9004, 4, 2, List(1000, 900, 100))), ord1002, List(Mumbai, India, 400064, Maharastra, Malad))"


In [0]:
orders_df3=orders_df2.select("filename",orders_df2.orders.customerId.alias('customerId'),orders_df2.orders.orderDate.alias('orderDate'),orders_df2.orders.orderDetails.alias('orderDetails'),orders_df2.orders.shipmentDetails.alias('shipmentDetails'))
display(orders_df3)

filename,customerId,orderDate,orderDetails,shipmentDetails
orderDetails,cust5001,2021-12-24 00.00.00.000,"List(List(prd9001, 2, 1, List(550, 500, 50)), List(prd9002, 3, 2, List(300, 240, 60)))","List(Delhi, India, 110040, New Delhi, M.G.Road)"
orderDetails,cust5002,2021-12-25 00.00.00.000,"List(List(prd9001, 1, 1, List(275, 250, 25)), List(prd9004, 4, 2, List(1000, 900, 100)))","List(Mumbai, India, 400064, Maharastra, Malad)"


In [0]:
orders_df4=orders_df3.select("filename","customerId","orderDate",explode(orders_df3.orderDetails),'shipmentDetails')
display(orders_df4)

filename,customerId,orderDate,col,shipmentDetails
orderDetails,cust5001,2021-12-24 00.00.00.000,"List(prd9001, 2, 1, List(550, 500, 50))","List(Delhi, India, 110040, New Delhi, M.G.Road)"
orderDetails,cust5001,2021-12-24 00.00.00.000,"List(prd9002, 3, 2, List(300, 240, 60))","List(Delhi, India, 110040, New Delhi, M.G.Road)"
orderDetails,cust5002,2021-12-25 00.00.00.000,"List(prd9001, 1, 1, List(275, 250, 25))","List(Mumbai, India, 400064, Maharastra, Malad)"
orderDetails,cust5002,2021-12-25 00.00.00.000,"List(prd9004, 4, 2, List(1000, 900, 100))","List(Mumbai, India, 400064, Maharastra, Malad)"


In [0]:
orders_df5=orders_df4.select("filename","customerId","orderDate",orders_df4.col.productId.alias('productId'), \
    orders_df4.col.quantity.alias('quantity'), \
        orders_df4.col.sequence.alias('sequence'), \
            orders_df4.col.totalPrice.alias('totalPrice'), \
            orders_df4.shipmentDetails.city.alias('city'), \
            orders_df4.shipmentDetails.country.alias('country'), \
            orders_df4.shipmentDetails.postalCode.alias('postalCode'), \
            orders_df4.shipmentDetails.state.alias('state'), \
            orders_df4.shipmentDetails.street.alias('street') )

display(orders_df5)

filename,customerId,orderDate,productId,quantity,sequence,totalPrice,city,country,postalCode,state,street
orderDetails,cust5001,2021-12-24 00.00.00.000,prd9001,2,1,"List(550, 500, 50)",Delhi,India,110040,New Delhi,M.G.Road
orderDetails,cust5001,2021-12-24 00.00.00.000,prd9002,3,2,"List(300, 240, 60)",Delhi,India,110040,New Delhi,M.G.Road
orderDetails,cust5002,2021-12-25 00.00.00.000,prd9001,1,1,"List(275, 250, 25)",Mumbai,India,400064,Maharastra,Malad
orderDetails,cust5002,2021-12-25 00.00.00.000,prd9004,4,2,"List(1000, 900, 100)",Mumbai,India,400064,Maharastra,Malad


In [0]:
orders_df6=orders_df5.select("filename","customerId","orderDate","productId","quantity","sequence",orders_df5.totalPrice.gross.alias("gross"), \
    orders_df5.totalPrice.net.alias("net"), \
        orders_df5.totalPrice.tax.alias("tax"), \
                             "city","country","postalCode","state","street")

display(orders_df6)

filename,customerId,orderDate,productId,quantity,sequence,gross,net,tax,city,country,postalCode,state,street
orderDetails,cust5001,2021-12-24 00.00.00.000,prd9001,2,1,550,500,50,Delhi,India,110040,New Delhi,M.G.Road
orderDetails,cust5001,2021-12-24 00.00.00.000,prd9002,3,2,300,240,60,Delhi,India,110040,New Delhi,M.G.Road
orderDetails,cust5002,2021-12-25 00.00.00.000,prd9001,1,1,275,250,25,Mumbai,India,400064,Maharastra,Malad
orderDetails,cust5002,2021-12-25 00.00.00.000,prd9004,4,2,1000,900,100,Mumbai,India,400064,Maharastra,Malad


In [0]:
df_fx=spark.read.option("multiline",True).json("/FileStore/tables/FXUSDCAD.json")
display(df_fx)

observations seriesDetail terms List(List(List(1.3435), 2017-01-03), List(List(1.3315), 2017-01-04), List(List(1.3244), 2017-01-05), List(List(1.3214), 2017-01-06), List(List(1.3240), 2017-01-09), List(List(1.3213), 2017-01-10), List(List(1.3250), 2017-01-11), List(List(1.3106), 2017-01-12), List(List(1.3141), 2017-01-13), List(List(1.3152), 2017-01-16), List(List(1.3057), 2017-01-17), List(List(1.3099), 2017-01-18), List(List(1.3303), 2017-01-19), List(List(1.3356), 2017-01-20), List(List(1.3291), 2017-01-23), List(List(1.3174), 2017-01-24), List(List(1.3088), 2017-01-25), List(List(1.3103), 2017-01-26), List(List(1.3123), 2017-01-27), List(List(1.3114), 2017-01-30), List(List(1.3030), 2017-01-31), List(List(1.3074), 2017-02-01), List(List(1.3016), 2017-02-02), List(List(1.3016), 2017-02-03), List(List(1.3099), 2017-02-06), List(List(1.3167), 2017-02-07), List(List(1.3157), 2017-02-08), List(List(1.3127), 2017-02-09), List(List(1.3084), 2017-02-10), List(List(1.3082), 2017-02-13), List(List(1.3070), 2017-02-14), List(List(1.3088), 2017-02-15), List(List(1.3055), 2017-02-16), List(List(1.3102), 2017-02-17), List(List(1.3148), 2017-02-21), List(List(1.3181), 2017-02-22), List(List(1.3110), 2017-02-23), List(List(1.3095), 2017-02-24), List(List(1.3118), 2017-02-27), List(List(1.3249), 2017-02-28), List(List(1.3338), 2017-03-01), List(List(1.3387), 2017-03-02), List(List(1.3405), 2017-03-03), List(List(1.3404), 2017-03-06), List(List(1.3417), 2017-03-07), List(List(1.3475), 2017-03-08), List(List(1.3513), 2017-03-09), List(List(1.3464), 2017-03-10), List(List(1.3447), 2017-03-13), List(List(1.3484), 2017-03-14), List(List(1.3428), 2017-03-15), List(List(1.3319), 2017-03-16), List(List(1.3337), 2017-03-17), List(List(1.3358), 2017-03-20), List(List(1.3317), 2017-03-21), List(List(1.3360), 2017-03-22), List(List(1.3336), 2017-03-23), List(List(1.3374), 2017-03-24), List(List(1.3373), 2017-03-27), List(List(1.3374), 2017-03-28), List(List(1.3364), 2017-03-29), List(List(1.3304), 2017-03-30), List(List(1.3310), 2017-03-31), List(List(1.3384), 2017-04-03), List(List(1.3425), 2017-04-04), List(List(1.3409), 2017-04-05), List(List(1.3418), 2017-04-06), List(List(1.3395), 2017-04-07), List(List(1.3349), 2017-04-10), List(List(1.3332), 2017-04-11), List(List(1.3300), 2017-04-12), List(List(1.3275), 2017-04-13), List(List(1.3289), 2017-04-17), List(List(1.3377), 2017-04-18), List(List(1.3460), 2017-04-19), List(List(1.3482), 2017-04-20), List(List(1.3506), 2017-04-21), List(List(1.3492), 2017-04-24), List(List(1.3592), 2017-04-25), List(List(1.3593), 2017-04-26), List(List(1.3624), 2017-04-27), List(List(1.3662), 2017-04-28), List(List(1.3660), 2017-05-01), List(List(1.3720), 2017-05-02), List(List(1.3714), 2017-05-03), List(List(1.3743), 2017-05-04), List(List(1.3712), 2017-05-05), List(List(1.3705), 2017-05-08), List(List(1.3728), 2017-05-09), List(List(1.3672), 2017-05-10), List(List(1.3707), 2017-05-11), List(List(1.3714), 2017-05-12), List(List(1.3641), 2017-05-15), List(List(1.3596), 2017-05-16), List(List(1.3614), 2017-05-17), List(List(1.3611), 2017-05-18), List(List(1.3544), 2017-05-19), List(List(1.3490), 2017-05-23), List(List(1.3460), 2017-05-24), List(List(1.3453), 2017-05-25), List(List(1.3456), 2017-05-26), List(List(1.3446), 2017-05-29), List(List(1.3469), 2017-05-30), List(List(1.3500), 2017-05-31), List(List(1.3500), 2017-06-01), List(List(1.3504), 2017-06-02), List(List(1.3483), 2017-06-05), List(List(1.3461), 2017-06-06), List(List(1.3492), 2017-06-07), List(List(1.3503), 2017-06-08), List(List(1.3453), 2017-06-09), List(List(1.3415), 2017-06-12), List(List(1.3238), 2017-06-13), List(List(1.3209), 2017-06-14), List(List(1.3283), 2017-06-15), List(List(1.3232), 2017-06-16), List(List(1.3220), 2017-06-19), List(List(1.3267), 2017-06-20), List(List(1.3310), 2017-06-21), List(List(1.3241), 2017-06-22), List(List(1.3268), 2017-06-23), List(List(1.3238), 2017-06-26), List(List(1.3188), 2017-06-27), List(List(1.3060), 2017-06-

In [0]:
from pyspark.sql.functions import explode
obser_df=df_fx.select(explode(df_fx.observations).alias("obsrv"))
display(obser_df)

obsrv
"List(List(1.3435), 2017-01-03)"
"List(List(1.3315), 2017-01-04)"
"List(List(1.3244), 2017-01-05)"
"List(List(1.3214), 2017-01-06)"
"List(List(1.3240), 2017-01-09)"
"List(List(1.3213), 2017-01-10)"
"List(List(1.3250), 2017-01-11)"
"List(List(1.3106), 2017-01-12)"
"List(List(1.3141), 2017-01-13)"
"List(List(1.3152), 2017-01-16)"


In [0]:
final_df=obser_df.select(obser_df.obsrv.FXUSDCAD.v.alias("version"),obser_df.obsrv.d.alias("date") )
display(final_df)

version,date
1.3435,2017-01-03
1.3315,2017-01-04
1.3244,2017-01-05
1.3214,2017-01-06
1.3240,2017-01-09
1.3213,2017-01-10
1.3250,2017-01-11
1.3106,2017-01-12
1.3141,2017-01-13
1.3152,2017-01-16
